In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [27]:
# ! pip install -q kaggle
# from google.colab import files
# files.upload()

In [28]:
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json

In [29]:
# ! kaggle competitions download -c foursquare-location-matching

In [30]:
# ! mkdir location_matching

In [31]:
# ! unzip foursquare-location-matching.zip -d location_matching 

In [32]:
# from google.colab import drive
# drive.mount('/content/drive')

In [33]:
# import pandas as pd

# train = pd.read_csv("/content/location_matching/pairs.csv")
train = pd.read_csv('../input/foursquare-location-matching/pairs.csv')

In [34]:
! pip install python-Levenshtein

In [35]:
import numpy as np
import Levenshtein as lev
from math import radians, cos, sin, asin, sqrt
from sklearn.utils import shuffle

In [36]:

def distance_compute(lat1, lat2, lon1, lon2): 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 
    return c * r

In [37]:
def category_match_ratio(c1, c2):
  count = 0
  categories1 = c1.split(",")
  categories2 = c2.split(",")
  for c1 in categories1:
    for c2 in categories2:
      if(c1.replace(" ", "").lower() == c2.replace(" ", "").lower()):
        count+=1
  return count*2/(len(categories1)+len(categories2))

In [38]:
def lev_ratio(s1, s2):
  if(s1 != "nan" and s2 != "nan"):
    return lev.ratio(s1.lower(), s2.lower())
  else:
    return np.nan
def is_match(s1, s2):
  if(s1 != "nan" and s2 != "nan"):
    if(s1.lower() == s2.lower()):
      return 1
    else:
      return 0
  else:
    return np.nan

In [39]:
%load_ext Cython

In [40]:
%%cython
def LCS(str S, str T):
    cdef int i, j
    cdef list dp = [[0] * (len(T) + 1) for _ in range(len(S) + 1)]
    for i in range(len(S)):
        for j in range(len(T)):
            dp[i + 1][j + 1] = max(dp[i][j] + (S[i] == T[j]), dp[i + 1][j], dp[i][j + 1], dp[i + 1][j + 1])
    return dp[len(S)][len(T)]

In [41]:
def lcs(s1, s2):
  if(s1 != "nan" and s2 != "nan"):
    return LCS(s1, s2)
  else:
    return np.nan

In [42]:
train_match = train[train["match"] == True]
train_not_match = train[train["match"] == False]

In [43]:
cutted_train_match = train_match.iloc[:int(len(train_match)/2)]

In [44]:
del train

In [45]:
train = shuffle(pd.concat([train_not_match, cutted_train_match], axis=0, ignore_index=True)).reset_index()

In [46]:
train["match"].value_counts()

In [47]:
dist_endure = 1.5
feature_chcek = 0.5

In [48]:
train_feature = ["distance", "name_lev_ratio", "name_lcs", "address_lev_ratio", "address_lcs", "city",  "state", "country", "url", "phone", "category_match_ratio"]
train_x = []
train_y = []
twelve_count = 0
for index in range(len(train)):
  row = train.iloc[index]
  train_row = []
  check_list = []
  dis = distance_compute(row["latitude_1"], row["latitude_2"], row["longitude_1"], row["longitude_2"])
  if(dis <= dist_endure):
    row = row.astype("str")
    train_row.append(dis/dist_endure)
    train_row.append(lev.ratio(row["name_1"], row["name_2"]))
    check_list.append(train_row[-1])
    train_row.append(lcs(row["name_1"], row["name_2"]))

    train_row.append(lev_ratio(row["address_1"], row["address_2"]))
    check_list.append(train_row[-1])
    train_row.append(lcs(row["address_1"], row["address_2"]))
    
    train_row.append(is_match(row["city_1"], row["city_2"]))
    check_list.append(train_row[-1])

    train_row.append(is_match(row["state_1"], row["state_2"]))
    check_list.append(train_row[-1])

    train_row.append(is_match(row["country_1"], row["country_2"]))

    train_row.append(is_match(row["url_1"], row["url_2"]))

    train_row.append(is_match(row["phone_1"], row["phone_2"]))

    train_row.append(category_match_ratio(row["categories_1"], row["categories_2"]))

    if(max(check_list)>0.5):
      train_x.append(train_row)

      if(row["match"] == "True"):
        train_y.append(1)
      else:
        train_y.append(0)



In [49]:
X_train = np.array(train_x)
Y_train = np.array(train_y)

In [50]:
print('train_x:', X_train.shape)
print('train_y:', Y_train.shape)

In [51]:
np.unique(Y_train, return_counts=True)

In [52]:
!pip install lightgbm

In [53]:
import lightgbm as lgb

params = {
    'objective': "binary",
    'learning_rate': 0.01,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'max_depth': 7,   
    'num_leaves': 35, 
    'n_estimators': 1000, 
    "colsample_bytree": 0.9,
}

model = lgb.LGBMClassifier(**params)

In [54]:
model.fit(X_train,Y_train)

In [55]:
# pre = model.predict(X_train)

In [56]:
# np.unique(pre, return_counts=True)

In [57]:
# import json
# from datetime import datetime, timezone, timedelta


# with open('/content/drive/MyDrive/Foursquare - Location Matching/record.json') as f:
#     record = json.load(f)
# current_number = record["number"]

In [58]:
# current_number += 1 
# model.booster_.save_model(f'/content/drive/MyDrive/Foursquare - Location Matching/lgb_{current_number}.txt')
model.booster_.save_model('lgb_model.txt')

In [59]:
# model_detail = params
# model_detail["inputs"] = train_feature
# model_detail["dist_endure"] = dist_endure
# model_detail["train_balance"] = "balanced"
# model_detail["feature_check"] = feature_chcek
# with open(f'/content/drive/MyDrive/Foursquare - Location Matching/params/params_{current_number}.json', "w") as f:
#   json.dump(params, f)